In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

['Hindi_English_Truncated_Corpus.csv']


In [72]:
lines=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [73]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [74]:
lines=lines[lines['source']=='ted']

In [75]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,"to turn on the lights or to bring him a glass of water,","लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [76]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [77]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [78]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [79]:
lines=lines.sample(n=30000,random_state=42)
lines.shape

(30000, 3)

In [80]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [81]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [82]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [83]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [84]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [85]:
lines.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है _END
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate climate solutions into all of your innovations,START_ और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें _END


In [86]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [87]:
len(all_eng_words)

15348

In [88]:
all_eng_words

{'“belief”',
 'holocaust',
 'humphrey',
 'figure',
 'cozy',
 'sisters',
 '“could',
 'imu',
 'salsa',
 'economics',
 'shelf',
 '“whos',
 'creators',
 'kangaroo',
 'geography',
 'medicaid',
 'accounting',
 'reevaluate',
 'embody',
 'consulted',
 'bicycle',
 'single',
 'cloths',
 'simpler',
 'juggler',
 'mecca',
 '“accelerator',
 'input',
 'कोई',
 'forbidding',
 'luanda',
 'changes',
 'clinics',
 'damaged',
 'ombudsman',
 'cashed',
 'masking',
 'journeys',
 'gentleman',
 'hit',
 'endure',
 'highline',
 'spectators',
 'spryness',
 'peach”',
 'together”',
 '“admiration',
 'world”',
 'too',
 'preprogrammed',
 'threeandahalf',
 'victorian',
 'sweettalking',
 'वह',
 'licenser',
 '“education',
 'phrases',
 'disciplines',
 'half',
 'syrians',
 'wielding',
 'mysore',
 'nearly',
 'desolate',
 'novogratz',
 'puzzlement',
 'musical',
 'literally”',
 'bookshop',
 'copout',
 'voted',
 'darkness',
 'claims',
 'taxonomy',
 'chance',
 'presidential',
 'stored',
 'displaced',
 'purposed',
 'insiders',
 'm

In [89]:
len(all_hindi_words)

19406

In [90]:
all_hindi_words

{'स्वरुप',
 'हँसी',
 'एडिनाइकी',
 'भौतिकताओं',
 'ढ़ाँचे',
 'कमी',
 'सफ़लताओं',
 'तीनआयामी',
 'पकडना',
 'आपका',
 'रहें।',
 'प्रायोगिक',
 'पहले।',
 'उत्कर्ष',
 'कर्तव्यों',
 'imu',
 'दिखेगी।',
 'भटक',
 'धारियों',
 'संबंधों',
 'डेसिमल',
 'दौड़ना',
 'ओकोलोमा',
 'सिखाओगी',
 'कोई',
 'बोलने',
 'एरिअल',
 'हीं',
 'उच्चतर',
 'घनत्व',
 'गिरती',
 'टेबिल',
 'हमें।',
 'बसाने',
 'रवांडा',
 'दिखेगा।',
 'साइबेरीयाई',
 'किमेल',
 'दर्शकएम्बार्क',
 'highline',
 'इच्छाएं',
 'सामान्य',
 'दरवाज़ा',
 'peach”',
 'निपटाने',
 'रेसिंगrolex',
 '“विश्व',
 'विष',
 'व्यंगछवियों',
 'व्यस्क्ता',
 'रेगिस्थान',
 'महत्वकांक्षाओं',
 'टपक',
 'उनसे',
 'विमर्श',
 'सांसों',
 'साओरी',
 'शुन्य',
 '“देखा',
 'विद',
 'वह',
 'लिपिक',
 'दिखावटी',
 'लोकमत',
 'औषधि',
 'हिला',
 'पहुँचाए',
 '“काश',
 'पिछले',
 'बदतर',
 'वाट',
 'अनैतिक',
 'मोटापे',
 'मरते',
 'सूचना',
 'संचार',
 'फायरवॉल',
 'राशि',
 'वार्ताकारों',
 'सौदेबाजी',
 'घिरनी',
 'सहभागियों',
 'बंगाली',
 'सरहद',
 'आंकने',
 'शाबाश',
 'विकर्षण',
 'ट्रैक',
 'जीवनचक्र',
 'बीजिंग',
 'सरल

In [91]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [92]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है _END,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate climate solutions into all of your innovations,START_ और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें _END,16,20


In [93]:
lines[lines['length_eng_sentence']>30].shape

(0, 5)

In [94]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [95]:
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है _END,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate climate solutions into all of your innovations,START_ और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें _END,16,20
79517,ted,and saw the demo by jeff han,START_ और जेफ़ हान द्वारा प्रदर्शन देखा होगा _END,7,9
23089,ted,this baby is fully electric,START_ यह बच्चा पूरी तरह से बिजली से चलता है _END,5,11
116699,ted,kids have no or very little say in making the rules,START_ फिर भी बच्चों को नियम बनाने का बिलकुल नहीं या बहुत कम मौका मिलता है _END,11,17
64132,ted,im going to add a little bit to my description of aging,START_ मै अपने द्वारा दिए गए उम्र के बढ़्ने के विवर्ण की कुछ व्याख्या करूंगा _END,12,16
6311,ted,by making it seamless,START_ इसे निरंतर बना कर _END,4,6


In [96]:
lines.shape

(29743, 5)

In [97]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [98]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [99]:
print(max_length_src)

20


In [100]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(15348, 19406)

In [101]:
num_decoder_tokens += 1 #for zero padding


In [102]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [103]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [104]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
79515,ted,your coach told you you were playing like a girl”,START_ तुम्हारे कोच ने कहा कि तुम लड़की की तरह खेल रहे हो” _END,10,14
54069,ted,effectively what youre saying to them is,START_ वास्तव में आप उनसे कह रहे हैं _END,7,9
102660,ted,“no no no the only thing with him is hes an idiot,START_ “नहीं नहीं नहीं तकलीफ बस यह है कि वो मूर्ख हैं _END,12,13
17877,ted,and the khan academy videos are there,START_ और ये ख़ान अकादमी के विडियो हैं। _END,7,9
97237,ted,that when they realize that vulnerability and tenderness are important,START_ कि जब उन्हें पता चलता है कि अतिसंवेदनशीलता और कोमलता महत्वपूर्ण हैं _END,10,14
45268,ted,in fact if the world were just filled with compassion,START_ वास्तव में यदि संसार मात्र करुणा से भरा होता _END,10,11
19546,ted,which was by not a lot,START_ कुछ खास ऊपर नही _END,6,6
6302,ted,and at the same time,START_ और ठीक उसी समय _END,5,6
82516,ted,but this one probably smells pretty much like it,START_ मगर इसकी दुर्गन्ध कुछ वैसी ही होती होगी _END,9,10
23778,ted,and our relationships with other people,START_ समय और हमारे अन्य लोगों के साथ सम्बन्ध। _END,6,10


### Split the data into train and test

In [105]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((23794,), (5949,))

In [106]:
X_train

99917     and this is something that grows in complexity                             
74681     entrepreneurs are people because we have a lot of them in this room        
60899     but rather it devotes this revenue                                         
57733     versebyverse                                                               
80922     with the confluence of fear ethnic strife                                  
22596     i have listened to the three previous speakers                             
94697     reweaving the urban fabric that we already have                            
109906    what are you most passionate about                                         
72194     its good for motivated students                                            
39116     i jump out next to her going “dont you move”                               
43528     so based on that success the following year                                
12932     the large majority of men                   

In [107]:
y_train

99917     START_ और ये कुछ ऐसा है कि जैसेजैसे लोग इसका प्रयोग करते हैं तो ये जटिलता में बढ़ता है _END       
74681     START_ उद्यमी वो लोग होते है और ऐसे कई लोग हमारे साथ आज इस कमरे में है _END                       
60899     START_ बल्कि इस आय को वह _END                                                                     
57733     START_ एक एक आयत _END                                                                             
80922     START_ जातिआधारित विवादों फ़ैलते डर के माहौल और _END                                              
22596     START_ पहले के तीन वक्ताओं की बातें मैंने सुनी _END                                               
94697     START_ उसी शहरी ढाँचे में फेरबदल करने की जो हमारे पास है _END                                     
109906    START_ आप सबसे अधिक किस के बारे में भावुक हो _END                                                 
72194     START_ ये बस प्रेरित और उत्सुक छात्रों के लिये ही ठीक हैं। _END                                   
39116     START_ मै

### Let us save this data

In [108]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [109]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [110]:
latent_dim=300

In [111]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [112]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [113]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [114]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    4604400     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 300)    5822100     input_4[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LS

In [115]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [116]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Epoch 1/100
185/185 [==============================] - 85s 459ms/step - loss: 6.3844 - val_loss: 6.0623
Epoch 2/100
185/185 [==============================] - 81s 440ms/step - loss: 5.7499 - val_loss: 5.6924
Epoch 3/100
185/185 [==============================] - 82s 443ms/step - loss: 5.3901 - val_loss: 5.5012
Epoch 4/100
185/185 [==============================] - 82s 443ms/step - loss: 5.1450 - val_loss: 5.4031
Epoch 5/100
185/185 [==============================] - 84s 453ms/step - loss: 4.9331 - val_loss: 5.3289
Epoch 6/100
185/185 [==============================] - 82s 444ms/step - loss: 4.7399 - val_loss: 5.2631
Epoch 7/100
185/185 [==============================] - 82s 441ms/step - loss: 4.5636 - val_loss: 5.2250
Epoch 8/100
185/185 [==============================] - 82s 441ms/step - loss: 4.3977 - val_loss: 5.1944
Epoch 9/100
185/185 [==============================] - 82s 442ms/step - loss: 4.2420 - val_loss: 5.1812
Epoch 10/100
185/185 [==============================] - 82s 443m

In [117]:
model.save_weights('nmt_weights.h5')

In [118]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [119]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [120]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [121]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and this is something that grows in complexity
Actual Hindi Translation:  और ये कुछ ऐसा है कि जैसेजैसे लोग इसका प्रयोग करते हैं तो ये जटिलता में बढ़ता है 
Predicted Hindi Translation:  और ये कुछ ऐसा है कि ये लोग इसका अनुभव है कि 


In [122]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: entrepreneurs are people because we have a lot of them in this room
Actual Hindi Translation:  उद्यमी वो लोग होते है और ऐसे कई लोग हमारे साथ आज इस कमरे में है 
Predicted Hindi Translation:  उद्यमी वो लोग थे की है और इस के महान हैं। 


In [123]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: but rather it devotes this revenue
Actual Hindi Translation:  बल्कि इस आय को वह 
Predicted Hindi Translation:  बल्कि इस आय की वह मैं 


In [124]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: versebyverse
Actual Hindi Translation:  एक एक आयत 
Predicted Hindi Translation:  एक एक आयत 


In [125]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: with the confluence of fear ethnic strife
Actual Hindi Translation:  जातिआधारित विवादों फ़ैलते डर के माहौल और 
Predicted Hindi Translation:  स्थान पर ध्यान दोस्तों 


In [138]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i have listened to the three previous speakers
Actual Hindi Translation:  पहले के तीन वक्ताओं की बातें मैंने सुनी 
Predicted Hindi Translation:  मैंने तीन सौ करोड़ राजनीति को पहले दिया गया है 


In [1]:
from nltk.translate import bleu_score

def calculate_bleu_score(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    weights = (0.25, 0.25, 0.25, 0.25)
    bleu = bleu_score.sentence_bleu(reference, candidate, weights=weights)
    return bleu

bleu_scores = []
for i in range(len(target_array)):
    reference = target_array[i]
    candidate = predict_array[i]
    bleu_score_i = calculate_bleu_score(reference, candidate)
    bleu_scores.append(bleu_score_i)

average_bleu_score = np.mean(bleu_scores)
print(f'Average BLEU Score: {average_bleu_score}')

Average BLEU Score: 23.01307651923
